# 신장의무기록사본 OCR 서비스 구현

## 1. 훈련용 데이터 만들기

### 1-1. 라이브러리 로드

In [8]:
import cv2
import numpy as np
import pandas as pd
from glob import glob
import json
import math
import os
os.chdir('D:/kidney/source')

import tensorflow as tf
import efficientnet.tfkeras
from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import ModelCheckpoint

### 1-2. 이미지 로드
### 1-3. labelme_json 파일 읽기

In [9]:
image = cv2.imread('../image/SCAN_01.jpg')

with open('../image/SCAN_01.json', "r", encoding='UTF8') as scan_json:
    json_dict = json.load(scan_json)

#json_dict

### 1-4. 이미지 크기 통일 함수

In [10]:
def border_make(image):
    row, col = image.shape[:2]
    bottom = image[row-2:row, 0:col]
    mean = cv2.mean(bottom)[0]
    
    col_bordersize = (55-col)/2
    row_bordersize = (55-row)/2
    
    border = cv2.copyMakeBorder(
        image,
        top = math.ceil(row_bordersize),
        bottom = math.floor(row_bordersize),
        left = math.ceil(col_bordersize),
        right = math.floor(col_bordersize),
        borderType = cv2.BORDER_ISOLATED, #BORDER_ISOLATED  BORDER_CONSTANT
        value = [mean, mean, mean]
    )
    return border

json_dict['shapes'][0]['label'] #0만 바꿔주면 됨.

### 1-5. Y_train의 컬럼명(=label) 추출

In [11]:
data = json_dict['shapes'] 

In [12]:
Y_train = []
for i in range(len(data)):
    label = json_dict['shapes'][i]['label']
    Y_train.append(label)

In [13]:
Y_train

['의',
 '뢰',
 '일',
 ':',
 '2',
 '0',
 '0',
 '9',
 '-',
 '0',
 '6',
 '-',
 '0',
 '8',
 '접',
 '수',
 '일',
 ':',
 '2',
 '0',
 '1',
 '9',
 '-',
 '1',
 '1',
 '-',
 '0',
 '7',
 '오',
 '전',
 '7',
 ':',
 '1',
 '9',
 ':',
 '0',
 '5',
 '결',
 '과',
 '보',
 '고',
 '일',
 ':',
 '2',
 '0',
 '1',
 '9',
 '-',
 '1',
 '1',
 '1',
 '0',
 '7',
 '페',
 '이',
 '지',
 '3',
 '/',
 '4',
 '검',
 '사',
 '명',
 '결',
 '과',
 '상',
 '태',
 '참',
 '고',
 '치',
 '단',
 '위',
 '검',
 '사',
 '명',
 '결',
 '과',
 '상',
 '태',
 '참',
 '고',
 '치',
 '단',
 '위',
 '검',
 '사',
 '명',
 '결',
 '과',
 '상',
 '태',
 '참',
 '고',
 '치',
 '단',
 '위',
 '5',
 '.',
 '6',
 '6',
 '4',
 '.',
 '9',
 '2',
 '1',
 '4',
 '.',
 '5',
 '4',
 '3',
 '.',
 '4',
 '8',
 '8',
 '.',
 '2',
 '2',
 '9',
 '.',
 '5',
 '3',
 '3',
 '.',
 '4',
 '2',
 '0',
 '6',
 '1',
 '1',
 '.',
 '7',
 '9',
 '.',
 '4',
 'W',
 'B',
 'C',
 'R',
 'B',
 'C',
 'H',
 'b',
 'H',
 'c',
 't',
 'M',
 'C',
 'V',
 'M',
 'C',
 'H',
 'M',
 'C',
 'H',
 'C',
 'P',
 'L',
 'T',
 'R',
 'D',
 'W',
 'P',
 'D',
 'W',
 '4',
 '.',
 '0',
 '1'

### 1-6. 훈련용 이미지 reshape

In [14]:
#image = cv2.imread('../image/SCAN_01.jpg', cv2.IMREAD_COLOR) #위에서 정의했음.
img_lst =[]
for i in range(len(json_dict['shapes'])):
    # 첫 좌표
    x1 = int(json_dict['shapes'][i]['points'][0][0]) #0,0 첫좌표의 행값
    y1 = int(json_dict['shapes'][i]['points'][0][1]) #0,0 첫좌표의 열값
    # 마지막 좌표
    x2 = int(json_dict['shapes'][i]['points'][1][0])
    y2 = int(json_dict['shapes'][i]['points'][1][1])
    
    #크롭 이미지
    if x1 > x2:
        if y1 > y2:
            cropped_image = image[y2: y1, x2: x1].copy()    
        else:
            cropped_image = image[y1: y2, x2: x1].copy()    
    else:
        if y1 > y2:
            cropped_image = image[y2: y1, x1: x2].copy()    
        else:
            cropped_image = image[y1: y2, x1: x2].copy()    

    #이미지 사이즈 통일 (55*55) 
    img = border_make(cropped_image)
    # 이미지 그레이스케일 적용
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 이미지 트레시 홀드 적용
    ret ,img = cv2.threshold(dst,127,255,0)
    
    #SCAN_01.PNG의 모든 글자를 4차원의 형태로 만들기
    img_lst.append(img)
    X_train = np.array(img_lst).reshape(-1,55,55,1)    # (971, 55, 55, 1)


np.save('../data/X_train', img_array, allow_pickle=True, fix_imports=True)

## 2. CNN OCR 모델 만들기

### 2-1. 독립변수, 종속변수 정의

In [15]:
X_train 
Y_train = pd.get_dummies(Y_train)
print(X_train.shape, Y_train.shape)

(971, 55, 55, 1) (971, 122)


In [16]:
X_train

array([[[[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        ...,

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]]],


       [[[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [

In [ ]:
Y_train

### 2-1-1. 예측을 위해 데이터 저장

In [8]:
df = pd.DataFrame(Y_train.columns)

In [10]:
df

,0
0,%
1,(
2,)
3,","
4,-
...,...
117,한
118,합
119,호
120,화


df.to_csv('../data/Y_train_unique.csv', index=False, encoding='utf-8-sig')

### 2-2. 모델 만들기

In [17]:
X = tf.keras.layers.Input(shape=[55, 55, 1])
H = tf.keras.layers.Conv2D(3, kernel_size = 5, activation = 'swish')(X)
H = tf.keras.layers.MaxPool2D()(H)
H = tf.keras.layers.Conv2D(6, kernel_size = 5, activation = 'swish')(H)
H = tf.keras.layers.MaxPool2D()(H)
H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84, activation = 'swish')(H)
Y = tf.keras.layers.Dense(122, activation = 'softmax')(H)

model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### 2-3. 모델 FIT

In [18]:
model.fit(X_train, Y_train, epochs=25) #, steps_per_epoch = 20)

Train on 971 samples
Epoch 1/25
971/971 [==============================] - 1s 878us/sample - loss: 73.9376 - acc: 0.0587
Epoch 2/25
971/971 [==============================] - 1s 747us/sample - loss: 5.0958 - acc: 0.1020
Epoch 3/25
971/971 [==============================] - 1s 713us/sample - loss: 4.6615 - acc: 0.1483
Epoch 4/25
971/971 [==============================] - 1s 743us/sample - loss: 4.3946 - acc: 0.2060
Epoch 5/25
971/971 [==============================] - 1s 724us/sample - loss: 3.7588 - acc: 0.2678
Epoch 6/25
971/971 [==============================] - 1s 745us/sample - loss: 3.1831 - acc: 0.3543
Epoch 7/25
971/971 [==============================] - 1s 730us/sample - loss: 2.5374 - acc: 0.4264
Epoch 8/25
971/971 [==============================] - 1s 760us/sample - loss: 2.0450 - acc: 0.5324
Epoch 9/25
971/971 [==============================] - 1s 734us/sample - loss: 1.5994 - acc: 0.6498
Epoch 10/25
971/971 [==============================] - 1s 790us/sample - loss: 1.3161 -

### 모델 저장

model.save('../model/OCR_ver1.0.h5')

### 2-4. 모델 요약

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 55, 55, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 51, 51, 3)         78        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 6)         456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 6)         0         
_________________________________________________________________
flatten (Flatten)            (None, 600)               0         
_________________________________________________________________
dense (Dense)                (None, 84)                50484 